In [1]:
import mne
from mne.time_frequency.multitaper import psd_array_multitaper
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import product
import sys

In [2]:
def eval_MItask(task, relax, ch, fmin, fmax):
    TaskRelax = []
    for epoch in [task, relax]:
        psdList = []
        for array in epoch.get_data()[:, epoch.ch_names.index(ch), :]:
            psd, freqs = psd_array_multitaper(array, epoch.info['sfreq'], fmin=fmin, fmax=fmax)
            psdList.append(psd)
        TaskRelax.append(np.median(psdList))
    return 100 * (TaskRelax[0] - TaskRelax[1]) / TaskRelax[1]

In [3]:
def sep_block(epoch, label, init):
    eventdf = pd.DataFrame(events, columns=['ind', 'step', 'id'])
    RestIndex = eventdf[eventdf.id == event_id['rest']].ind.values
    
    BlockList = []
    init = init
    for index in np.insert(RestIndex, len(RestIndex), eventdf[eventdf.id == event_id['training_finish']].ind.item()):
        BlockIndex = len(eventdf[(eventdf.id == event_id[label]) & (eventdf.ind < index)])
        BlockList.append(epoch[init:BlockIndex])
        init = BlockIndex
    
    return BlockList

In [4]:
def calc_block_psd(epochs, label, ch, init=0):
    epochList = sep_block(epochs, label, init)
    
    BlockPsdList = []
    for epoch in epochList:
        PsdList = []
        for array in epoch.get_data()[:, epoch.ch_names.index(ch), :]:
            psd, freqs = psd_array_multitaper(array, epoch.info['sfreq'], fmin=fmin, fmax=fmax)
            PsdList.append(psd)
        BlockPsdList.append(np.median(psd))

    return BlockPsdList

In [5]:
def create_MItest_erd_df(dfList, task, relax, label):
    for i in range(2): #left and right
        ch = 'C4' if i == 0 else 'C3'
        ERSP = eval_MItask(task[i], relax[i], ch, fmin, fmax)
        dfList[i] = dfList[i].append(pd.Series([label, ERSP], index=dfList[i].columns), ignore_index=True)

In [6]:
def create_block_erd_df(dfList, task, baseline):
    for i in range(2): #left and right
        if i == 0:
            ch = 'C4'
            taskLabel = 'task_left'
        else:
            ch = 'C3'
            taskLabel = 'task_right'
        block_psd = calc_block_psd(task[i], taskLabel, ch)
        for bNum, block in enumerate(['b1', 'b2', 'b3', 'b4', 'b5']):
            dfList[i] = dfList[i].append(pd.Series([block, 100*(block_psd[bNum]-baseline[i])/baseline[i]], index=dfList[i].columns), ignore_index=True)

In [7]:
fmin, fmax = 8, 13
MI_Df = [pd.DataFrame(columns=['timing', 'ERSP'])] * 2
Train_Df = [pd.DataFrame(columns=['block', 'ERSP'])] * 2

In [20]:
pid = 'sha1'
day = 'Day3'

In [21]:
raw = mne.io.read_raw_brainvision('result/' + pid + '_' + day + '.vhdr', preload=True)
#filter
raw.notch_filter(freqs=60)
raw.filter(l_freq=3, h_freq=50)
#re-refernce
re_signal = raw.get_data()[raw.ch_names.index('A2')] / 2
new_raw = raw.get_data()
new_raw[raw.ch_names.index('A2')] = re_signal
raw = mne.io.RawArray(new_raw, info=raw.info)
raw.set_eeg_reference(ref_channels=['A2'])
#epoch
events = mne.find_events(raw, stim_channel='TRIGGER')
trigger_table = pd.read_csv('trigger_table.csv')
trigger_table['sendnum'] = trigger_table['trigger'] * 256
event_id = trigger_table.set_index('label')['sendnum'].to_dict()
if day != 'Day1':
    for id in ['flandars', 'KVIQ', 'Mitest_pre', 'Mitest_pre_taskright', 'Mitest_pre_taskleft', 'Mitest_pre_relaxright', 'Mitest_pre_relaxleft']:
        del event_id[id]
epochs = mne.Epochs(raw, events, event_id, tmin=-1, tmax=4, detrend=1)
task = [epochs['task_left'], epochs['task_right']]
if day == 'Day1':
    Mitask_pre = [epochs['Mitest_pre_taskleft'], epochs['Mitest_pre_taskright']]
    Mirelax_pre = [epochs['Mitest_pre_relaxleft'], epochs['Mitest_pre_relaxright']]
Mitask_post = [epochs['Mitest_post_taskleft'], epochs['Mitest_post_taskright']]
Mirelax_post = [epochs['Mitest_post_relaxleft'], epochs['Mitest_post_relaxright']]

Extracting parameters from result/sha1_Day3.vhdr...
Setting channel info structure...
Reading 0 ... 1083163  =      0.000 ...  2166.326 secs...
Setting up band-stop filter from 59 - 61 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 59.35
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 59.10 Hz)
- Upper passband edge: 60.65 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 60.90 Hz)
- Filter length: 3301 samples (6.602 sec)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 3 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband e

In [22]:
baseline_df = pd.read_csv('result/' + pid + '_baseline_' + day + '.csv')
baseline = baseline_df.median().values[1:]*1e-12
baseline

array([5.18066388e-08, 3.96224587e-08])

In [23]:
if day == 'Day1':
    create_MItest_erd_df(MI_Df, Mitask_pre, Mirelax_pre, 'Day1_pre')
create_MItest_erd_df(MI_Df, Mitask_post, Mirelax_post, day + '_post')

Loading data for 15 events and 2501 original time points ...
0 bad epochs dropped
Loading data for 15 events and 2501 original time points ...
0 bad epochs dropped
Loading data for 15 events and 2501 original time points ...
0 bad epochs dropped
Loading data for 15 events and 2501 original time points ...
0 bad epochs dropped


In [24]:
create_block_erd_df(Train_Df, task, baseline)

Loading data for 10 events and 2501 original time points ...
0 bad epochs dropped
Loading data for 10 events and 2501 original time points ...
0 bad epochs dropped
Loading data for 10 events and 2501 original time points ...
0 bad epochs dropped
Loading data for 10 events and 2501 original time points ...
0 bad epochs dropped
Loading data for 10 events and 2501 original time points ...
0 bad epochs dropped
Loading data for 10 events and 2501 original time points ...
0 bad epochs dropped
Loading data for 10 events and 2501 original time points ...
0 bad epochs dropped
Loading data for 10 events and 2501 original time points ...
0 bad epochs dropped
Loading data for 10 events and 2501 original time points ...
0 bad epochs dropped
Loading data for 10 events and 2501 original time points ...
0 bad epochs dropped


In [25]:
MI_Df[1]

,timing,ERSP
0,Day1_pre,0.474753
1,Day1_post,-3.040719
2,Day2_post,7.564062
3,Day3_post,-0.298422


In [26]:
Train_Df[0]

,block,ERSP
0,b1,-74.081335
1,b2,-41.400170
2,b3,-33.975043
3,b4,-4.628198
4,b5,-2.941516
5,b1,-72.809450
6,b2,-77.054405
7,b3,-83.439846
8,b4,-82.635171
9,b5,-72.276651
